In [ ]:
!pip install pyscipopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 37.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import random
from scipy.ndimage import binary_dilation
from itertools import product
import math

In [ ]:
n = 100
KRATKA_M = 20
D_MIN_M = 50
BUFOR_M = 700
DROGA_SZER_M = 5

d_min = int(np.ceil(D_MIN_M / KRATKA_M))
bufor = max(1, round(BUFOR_M / KRATKA_M))
droga_szer = max(1, round(DROGA_SZER_M / KRATKA_M))

#siatka i droga
def dodaj_prostokat(siatka, sr, sc, wys, szer, kod=1):
    r_start = max(0, sr - wys // 2)
    r_end = min(n, sr + (wys + 1) // 2)
    c_start = max(0, sc - szer // 2)
    c_end = min(n, sc + (szer + 1) // 2)
    siatka[r_start:r_end, c_start:c_end] = kod

siatka = np.zeros((n, n))
for i in range(n):
    dodaj_prostokat(siatka, i, n // 2, 1, droga_szer, kod=2)

#budynki
budynki_m = [
    ((300, 850), (120, 180)),
    ((1600, 800), (100, 100)),
    ((1500, 1200), (80, 120)),
    ((1200, 1200), (90, 90)),
    ((200, 1200), (100, 120)),
    ((600, 800), (120, 80)),
    ((800, 800), (100, 120)),
    ((600, 1200), (120, 80)),
    ((800, 1200), (100, 120)),
]
for (sr_m, sc_m), (wys_m, szer_m) in budynki_m:
    sr = sr_m // KRATKA_M
    sc = sc_m // KRATKA_M
    wys = max(1, wys_m // KRATKA_M)
    szer = max(1, szer_m // KRATKA_M)
    dodaj_prostokat(siatka, sr, sc, wys, szer, kod=1)

#bufor i pola dostępne
przeszkody = (siatka != 0)
bufor_mask = binary_dilation(przeszkody, iterations=bufor)
puste_pola = [(i, j) for i in range(n) for j in range(n) if not bufor_mask[i, j]]

#funkcje chłodzenia
def cooling_linear(T, alpha=1):
    return max(0.01, T - alpha)

def cooling_exponential(T, beta=0.95):
    return T * beta

def cooling_reciprocal(T0, k, alpha=0.1):
    return T0 / (1 + alpha * k)

def cooling_logarithmic(T0, k, c=2):
    return T0 / math.log(k + c)

#tutaj te kratki nieszczesne
def odleglosc_w_kratkach(p1, p2):
    return np.hypot(p1[0] - p2[0], p1[1] - p2[1])

def ocena(turbiny):
    wynik = []
    for t in turbiny:
        if all(odleglosc_w_kratkach(t, istniejąca) >= d_min for istniejąca in wynik):
            wynik.append(t)
    return wynik

#funkcja wyżarzania
def run_sa(T0, cooling_type, m_n, puste_pola, seed=None):
    if seed is not None:
        random.seed(seed)
    obecne = []
    najlepsze = []
    T = T0
    epoka = 0
    max_epok = 50


    while T > 0.1 and epoka < max_epok:
        for _ in range(m_n):
            kandydat = obecne.copy()
            if len(obecne) < 20:
                weights = [0.8, 0.15, 0.05]
            else:
                weights = [0.4, 0.4, 0.2]

            operacja = random.choices(['dodaj', 'zamień', 'usuń'], weights=weights, k=1)[0]

            if operacja == 'dodaj' and len(kandydat) < len(puste_pola):
                nowa = random.choice(puste_pola)
                if all(odleglosc_w_kratkach(nowa, t) >= d_min for t in kandydat):
                    kandydat.append(nowa)

            elif operacja == 'zamień' and kandydat:
                i = random.randint(0, len(kandydat) - 1)
                nowa = random.choice(puste_pola)
                if all(odleglosc_w_kratkach(nowa, t) >= d_min for j, t in enumerate(kandydat) if j != i):
                    kandydat[i] = nowa

            elif operacja == 'usuń' and kandydat:
                i = random.randint(0, len(kandydat) - 1)
                kandydat.pop(i)

            poprawne = ocena(kandydat)

            if len(poprawne) >= len(najlepsze) or random.random() < np.exp((len(poprawne) - len(obecne)) / T):
                obecne = poprawne
                if len(poprawne) > len(najlepsze):
                    najlepsze = poprawne

        epoka += 1


        if cooling_type == "lin":
            T = cooling_linear(T, alpha=1)
        elif cooling_type == "exp":
            T = cooling_exponential(T, beta=0.95)
        elif cooling_type == "rec":
            T = cooling_reciprocal(T0, epoka, alpha=0.1)
        elif cooling_type == "log":
            T = cooling_logarithmic(T0, epoka, c=2)

    return len(najlepsze)

T0_options = [250, 500, 800, 1000]
cooling_options = ["lin", "exp", "rec", "log"]
m_n_options = list(range(1, 11))

results = []
for T0, cooling, m_n in product(T0_options, cooling_options, m_n_options):
    wyniki = []
    for _ in range(10):
        wynik = run_sa(T0, cooling, m_n, puste_pola)
        wyniki.append(wynik)
    avg = np.mean(wyniki)
    results.append((T0, cooling, m_n, avg))

top10 = sorted(results, key=lambda x: -x[3])[:10]
print("TOP 10 konfiguracji:")
for i, (T0, cooling, m_n, avg) in enumerate(top10, 1):
    print(f"{i}. T0={T0}, chłodzenie={cooling}, m_n={m_n} → średnia liczba turbin: {avg:.2f}")


TOP 10 konfiguracji:
1. T0=1000, chłodzenie=log, m_n=10 → średnia liczba turbin: 48.90
2. T0=250, chłodzenie=log, m_n=10 → średnia liczba turbin: 48.10
3. T0=500, chłodzenie=exp, m_n=10 → średnia liczba turbin: 48.00
4. T0=500, chłodzenie=log, m_n=10 → średnia liczba turbin: 48.00
5. T0=1000, chłodzenie=rec, m_n=10 → średnia liczba turbin: 47.10
6. T0=800, chłodzenie=exp, m_n=10 → średnia liczba turbin: 47.00
7. T0=250, chłodzenie=rec, m_n=10 → średnia liczba turbin: 46.70
8. T0=250, chłodzenie=exp, m_n=10 → średnia liczba turbin: 46.50
9. T0=1000, chłodzenie=exp, m_n=10 → średnia liczba turbin: 46.50
10. T0=250, chłodzenie=log, m_n=9 → średnia liczba turbin: 45.80
